# Fine-Tuning de Modelo LLM para Domínio Médico

Este notebook implementa o pipeline completo de fine-tuning de um modelo LLM para tarefas de question-answering médico baseado em evidências científicas.

## ⚠️ IMPORTANTE: Pré-requisito ANTES de usar este notebook

**Você PRECISA processar o dataset médico ANTES de usar este notebook!**

### Passo 1: Preparar Dataset (FAZER LOCALMENTE, ANTES DO COLAB)

No seu computador local, execute:

```bash
cd fine_tuning
python run_pipeline.py --all
```

Isso irá:
1. Processar o dataset médico (`ori_pqal.json`)
2. Anonimizar dados sensíveis
3. Formatar no padrão Alpaca
4. Gerar o arquivo: `formatted_medical_dataset.json`

### Passo 2: Usar no Google Colab

1. Faça upload do arquivo `formatted_medical_dataset.json` para o Colab (célula 4)
2. Execute todas as células sequencialmente
3. O modelo treinado será salvo e você pode fazer download

## Objetivos:
1. Carregar dataset médico formatado no padrão Alpaca
2. Carregar modelo base pré-quantizado (Unsloth)
3. Configurar LoRA para treinamento eficiente
4. Treinar modelo com dados médicos
5. Testar e salvar modelo treinado

## Requisitos:
- **Google Colab** (recomendado) ou ambiente com GPU
- GPU com pelo menos 8GB VRAM (recomendado 16GB+)
- Dataset formatado: `formatted_medical_dataset.json` (gerado pelo `run_pipeline.py`)

## Ordem de Execução:
Execute as células **sequencialmente** (de cima para baixo).


In [ ]:
# ============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS (OBRIGATÓRIO NO COLAB)
# ============================================================================
# Esta célula instala todas as bibliotecas necessárias para o fine-tuning.
# Execute esta célula primeiro se estiver usando Google Colab.

print("📦 Instalando dependências...")
print("⏳ Isso pode levar alguns minutos na primeira execução...")

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

print("\n✅ Dependências instaladas com sucesso!")


In [ ]:
# ============================================================================
# CÉLULA 2: IMPORTAÇÕES DE BIBLIOTECAS
# ============================================================================
# Importa todas as bibliotecas necessárias para o fine-tuning

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from pathlib import Path

print("✅ Bibliotecas importadas com sucesso!")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  GPU não detectada! Fine-tuning será muito lento em CPU.")


In [ ]:
# ============================================================================
# CÉLULA 3: CONFIGURAÇÕES COMPLETAS (TUDO INLINE)
# ============================================================================
# Todas as configurações estão definidas aqui - não precisa de arquivos externos

# ============================================================================
# CONFIGURAÇÕES DO MODELO BASE
# ============================================================================
MAX_SEQ_LENGTH = 2048
DTYPE = None  # Auto-detect
LOAD_IN_4BIT = True
DEFAULT_MODEL = "unsloth/llama-3-8b-bnb-4bit"

# ============================================================================
# CONFIGURAÇÕES LoRA
# ============================================================================
LORA_CONFIG = {
    "r": 16,
    "lora_alpha": 16,
    "lora_dropout": 0,
    "bias": "none",
    "target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    "use_gradient_checkpointing": "unsloth",
    "random_state": 3407,
    "use_rslora": False,
    "loftq_config": None,
}

# ============================================================================
# HIPERPARÂMETROS DE TREINAMENTO
# ============================================================================
TRAINING_CONFIG = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 5,
    "max_steps": 100,
    "learning_rate": 2e-4,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 3407,
    "output_dir": "outputs",
    "logging_steps": 1,
}

# ============================================================================
# CONFIGURAÇÕES DE DATASET
# ============================================================================
DATASET_CONFIG = {
    "dataset_num_proc": 2,
    "packing": False,
}

# ============================================================================
# CONFIGURAÇÕES DE INFERÊNCIA
# ============================================================================
INFERENCE_CONFIG = {
    "max_new_tokens": 256,
    "use_cache": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 50,
}

# ============================================================================
# FUNÇÕES DE PROMPTS (INLINE)
# ============================================================================
MEDICAL_ALPACA_PROMPT = """Below is a medical instruction that describes a task, paired with medical context and a question. Write a response that appropriately completes the request based on the provided medical evidence.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def get_medical_alpaca_prompt(instruction: str, input_text: str, response: str = "") -> str:
    """Formata um prompt médico usando o template Alpaca"""
    return MEDICAL_ALPACA_PROMPT.format(instruction, input_text, response)

def get_instruction_only() -> str:
    """Retorna apenas a instrução padrão para tarefas médicas"""
    return "Responda à pergunta baseando-se nos contextos fornecidos."

# ============================================================================
# CAMINHOS (AJUSTE PARA COLAB)
# ============================================================================
# No Colab, você pode:
# 1. Fazer upload do arquivo diretamente (use o caminho abaixo)
# 2. Montar Google Drive e usar caminho do Drive
# 3. Usar caminho absoluto do arquivo

FORMATTED_DATASET_PATH = Path("formatted_medical_dataset.json")  # Arquivo na raiz do Colab
MODEL_OUTPUT_DIR = Path("lora_model_medical")
TRAINING_OUTPUT_DIR = Path("outputs")

print("=" * 80)
print("CONFIGURAÇÕES DE FINE-TUNING")
print("=" * 80)
print(f"Modelo: {DEFAULT_MODEL}")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"LoRA rank: {LORA_CONFIG['r']}")
print(f"Learning rate: {TRAINING_CONFIG['learning_rate']}")
print(f"Max steps: {TRAINING_CONFIG['max_steps']}")
print(f"Dataset: {FORMATTED_DATASET_PATH}")
print(f"Output model: {MODEL_OUTPUT_DIR}")
print("=" * 80)


In [ ]:
# ============================================================================
# CÉLULA 4: UPLOAD DO DATASET FORMATADO
# ============================================================================
# ⚠️ IMPORTANTE: Este notebook espera que você já tenha processado o dataset!
#
# ANTES de usar este notebook, você DEVE ter executado localmente:
#   cd fine_tuning
#   python run_pipeline.py --all
#
# Isso gera o arquivo: formatted_medical_dataset.json
#
# Agora, faça upload desse arquivo para o Colab:

# Verifica se está no Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Verifica se o arquivo já existe
if not FORMATTED_DATASET_PATH.exists():
    if IN_COLAB:
        print("=" * 80)
        print("📤 UPLOAD DO DATASET FORMATADO")
        print("=" * 80)
        print("\n⚠️  Arquivo não encontrado!")
        print("\n📋 INSTRUÇÕES:")
        print("   1. Você deve ter processado o dataset ANTES de usar este notebook")
        print("   2. Execute localmente: python run_pipeline.py --all")
        print("   3. Isso gera: formatted_medical_dataset.json")
        print("   4. Agora faça upload desse arquivo abaixo:")
        print("\n" + "-" * 80)
        
        uploaded = files.upload()
        
        # Verifica se o arquivo foi enviado
        if 'formatted_medical_dataset.json' in uploaded:
            print("\n✅ Arquivo enviado com sucesso!")
        else:
            raise FileNotFoundError(
                "\n❌ Arquivo 'formatted_medical_dataset.json' não foi encontrado no upload.\n"
                "Certifique-se de que:\n"
                "  1. Você executou 'python run_pipeline.py --all' localmente\n"
                "  2. O arquivo gerado se chama exatamente 'formatted_medical_dataset.json'\n"
                "  3. Você fez upload do arquivo correto"
            )
    else:
        raise FileNotFoundError(
            f"\n❌ Dataset não encontrado: {FORMATTED_DATASET_PATH}\n\n"
            f"📋 INSTRUÇÕES:\n"
            f"   1. Execute localmente: python run_pipeline.py --all\n"
            f"   2. Isso gera: formatted_medical_dataset.json\n"
            f"   3. Coloque o arquivo no diretório atual ou ajuste FORMATTED_DATASET_PATH"
        )
else:
    print(f"✅ Arquivo já existe: {FORMATTED_DATASET_PATH}")

# ============================================================================
# CÉLULA 4.1: CARREGAMENTO DO DATASET FORMATADO
# ============================================================================
# Carrega o dataset médico já formatado no padrão Alpaca.
# Este arquivo foi gerado pelo pipeline run_pipeline.py --all

print("\n" + "=" * 80)
print("📦 CARREGANDO DATASET FORMATADO")
print("=" * 80)
print(f"Arquivo: {FORMATTED_DATASET_PATH}")

# load_dataset do Hugging Face carrega JSON diretamente
dataset = load_dataset("json", data_files=str(FORMATTED_DATASET_PATH), split="train")

print(f"\n✅ Dataset carregado: {len(dataset)} exemplos")
print(f"   Estrutura: {dataset.features}")

# Valida estrutura esperada (formato Alpaca)
expected_fields = ['instruction', 'input', 'output']
if not all(field in dataset.features for field in expected_fields):
    raise ValueError(
        f"❌ Dataset não está no formato Alpaca esperado!\n"
        f"   Campos esperados: {expected_fields}\n"
        f"   Campos encontrados: {list(dataset.features.keys())}\n"
        f"   Certifique-se de que executou 'python run_pipeline.py --all' corretamente."
    )

print(f"\n📄 Exemplo de entrada:")
print(f"   Instruction: {dataset[0]['instruction'][:100]}...")
print(f"   Input: {dataset[0]['input'][:100]}...")
print(f"   Output: {dataset[0]['output'][:100]}...")


In [ ]:
# ============================================================================
# CÉLULA 5: CARREGAMENTO DO MODELO BASE
# ============================================================================
# Carrega modelo pré-quantizado do Unsloth.
# Unsloth fornece modelos otimizados que reduzem uso de memória em ~75%
# mantendo qualidade próxima ao modelo original.

print("=" * 80)
print("CARREGANDO MODELO BASE")
print("=" * 80)
print("⏳ Isso pode levar alguns minutos na primeira execução...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=DEFAULT_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)

print("✅ Modelo carregado!")
print(f"   Parâmetros totais: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
# ============================================================================
# CÉLULA 6: CONFIGURAÇÃO LoRA
# ============================================================================
# LoRA (Low-Rank Adaptation) permite treinar apenas ~1-5% dos parâmetros,
# reduzindo drasticamente memória e tempo de treinamento.

print("=" * 80)
print("CONFIGURANDO LoRA")
print("=" * 80)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_CONFIG['r'],
    target_modules=LORA_CONFIG['target_modules'],
    lora_alpha=LORA_CONFIG['lora_alpha'],
    lora_dropout=LORA_CONFIG['lora_dropout'],
    bias=LORA_CONFIG['bias'],
    use_gradient_checkpointing=LORA_CONFIG['use_gradient_checkpointing'],
    random_state=LORA_CONFIG['random_state'],
    use_rslora=LORA_CONFIG['use_rslora'],
    loftq_config=LORA_CONFIG['loftq_config'],
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA configurado!")
print(f"   Parâmetros treináveis: {trainable_params:,}")
print(f"   Parâmetros totais: {total_params:,}")
print(f"   Fração treinável: {(trainable_params/total_params)*100:.2f}%")


In [ ]:
# ============================================================================
# CÉLULA 7: DEFINIÇÃO DO PROMPT ALPACA MÉDICO
# ============================================================================
# Define a função que formata exemplos do dataset para o formato Alpaca.
# Esta função será aplicada a cada exemplo durante o treinamento.

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    """
    Formata exemplos para o formato Alpaca médico
    
    Combina instruction, input e output em um único texto formatado
    que o modelo aprenderá a gerar.
    """
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Usa template Alpaca médico
        text = get_medical_alpaca_prompt(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

print("✅ Função de formatação definida!")


In [ ]:
# ============================================================================
# CÉLULA 8: PREPARAÇÃO DO DATASET PARA TREINAMENTO
# ============================================================================
# Aplica formatação de prompts a todos os exemplos do dataset

print("=" * 80)
print("FORMATANDO DATASET")
print("=" * 80)

formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

print(f"✅ Dataset formatado: {len(formatted_dataset)} exemplos")
print(f"   Estrutura: {formatted_dataset.features}")

# Mostra exemplo formatado
print(f"\nExemplo de texto formatado (primeiros 500 caracteres):")
print("-" * 80)
print(formatted_dataset[0]['text'][:500] + "...")
print("-" * 80)


In [ ]:
# ============================================================================
# CÉLULA 9: CONFIGURAÇÃO DO TRAINER
# ============================================================================
# Configura o SFTTrainer (Supervised Fine-Tuning Trainer) que gerencia
# todo o processo de treinamento.

print("=" * 80)
print("CONFIGURANDO TRAINER")
print("=" * 80)

training_args = TrainingArguments(
    per_device_train_batch_size=TRAINING_CONFIG['per_device_train_batch_size'],
    gradient_accumulation_steps=TRAINING_CONFIG['gradient_accumulation_steps'],
    warmup_steps=TRAINING_CONFIG['warmup_steps'],
    max_steps=TRAINING_CONFIG['max_steps'],
    learning_rate=TRAINING_CONFIG['learning_rate'],
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=TRAINING_CONFIG['logging_steps'],
    optim=TRAINING_CONFIG['optim'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    lr_scheduler_type=TRAINING_CONFIG['lr_scheduler_type'],
    seed=TRAINING_CONFIG['seed'],
    output_dir=str(TRAINING_OUTPUT_DIR),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=DATASET_CONFIG['dataset_num_proc'],
    packing=DATASET_CONFIG['packing'],
    args=training_args,
)

print("✅ Trainer configurado!")
print(f"   Batch efetivo: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Max steps: {training_args.max_steps}")
print(f"   Learning rate: {training_args.learning_rate}")


In [ ]:
# ============================================================================
# CÉLULA 10: TREINAMENTO DO MODELO
# ============================================================================
# Inicia o processo de treinamento. Este processo pode levar vários minutos
# ou horas dependendo do tamanho do dataset e performance da GPU.
#
# Durante o treinamento, você verá logs mostrando:
# - Loss (deve diminuir ao longo do tempo)
# - Learning rate atual
# - Progresso (steps completados)

print("=" * 80)
print("INICIANDO TREINAMENTO")
print("=" * 80)
print("⚠️  Este processo pode levar vários minutos ou horas...")
print("-" * 80)

trainer_stats = trainer.train()

print("\n" + "=" * 80)
print("✅ TREINAMENTO CONCLUÍDO")
print("=" * 80)
print(f"Loss final: {trainer_stats.training_loss:.4f}")
print(f"Steps completados: {trainer_stats.global_step}")


In [ ]:
# ============================================================================
# CÉLULA 11: TESTE DO MODELO TREINADO
# ============================================================================
# Testa o modelo com um exemplo médico para verificar a qualidade
# das respostas geradas.

print("=" * 80)
print("TESTANDO MODELO TREINADO")
print("=" * 80)

# Prepara modelo para inferência
FastLanguageModel.for_inference(model)

# Exemplo de teste
example_instruction = get_instruction_only()
example_input = """Contexto: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant produces perforations in its leaves through PCD.
Pergunta: Do mitochondria play a role in remodelling plant leaves during programmed cell death?"""

# Formata prompt (sem resposta, queremos que o modelo gere)
prompt = get_medical_alpaca_prompt(example_instruction, example_input, "")

# Tokeniza e gera
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer([prompt], return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=INFERENCE_CONFIG['max_new_tokens'],
    use_cache=INFERENCE_CONFIG['use_cache'],
    temperature=INFERENCE_CONFIG['temperature'],
    top_p=INFERENCE_CONFIG['top_p'],
    top_k=INFERENCE_CONFIG['top_k'],
)

generated_text = tokenizer.batch_decode(outputs)[0]

print("Prompt de entrada:")
print("-" * 80)
print(prompt[:300] + "...")
print("-" * 80)
print("\nResposta gerada:")
print("-" * 80)
print(generated_text)
print("-" * 80)


In [ ]:
# ============================================================================
# CÉLULA 12: SALVAMENTO DO MODELO
# ============================================================================
# Salva o modelo treinado (apenas adaptadores LoRA) e o tokenizer.
# O modelo salvo pode ser carregado depois para inferência.

print("=" * 80)
print("SALVANDO MODELO")
print("=" * 80)

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

model.save_pretrained(str(MODEL_OUTPUT_DIR))
tokenizer.save_pretrained(str(MODEL_OUTPUT_DIR))

print(f"✅ Modelo salvo em: {MODEL_OUTPUT_DIR}")
print("\nPara carregar o modelo depois, use:")
print(f"  model, tokenizer = FastLanguageModel.from_pretrained('{MODEL_OUTPUT_DIR}')")

# ============================================================================
# CÉLULA 12.1: DOWNLOAD DO MODELO (OPCIONAL - APENAS NO COLAB)
# ============================================================================
# No Google Colab, você pode fazer download do modelo treinado.
# Descomente as linhas abaixo para fazer download.

try:
    from google.colab import files
    import shutil
    
    print("\n" + "=" * 80)
    print("PREPARANDO DOWNLOAD DO MODELO")
    print("=" * 80)
    
    # Compacta o modelo
    shutil.make_archive('lora_model_medical', 'zip', str(MODEL_OUTPUT_DIR))
    
    # Faz download
    files.download('lora_model_medical.zip')
    
    print("✅ Download iniciado!")
except ImportError:
    print("\n💡 Para fazer download no Colab, descomente o código acima nesta célula.")


In [ ]:
# ============================================================================
# CÉLULA 14: UPLOAD PARA HUGGING FACE (OPCIONAL)
# ============================================================================
# Faz upload dos adaptadores LoRA para o Hugging Face.
# Isso permite usar o modelo via Inference API ou baixar depois.

print("=" * 80)
print("UPLOAD PARA HUGGING FACE")
print("=" * 80)

# Instala huggingface_hub se necessário
try:
    from huggingface_hub import HfApi, login, create_repo
    from huggingface_hub.utils import HfHubHTTPError
except ImportError:
    print("📦 Instalando huggingface_hub...")
    !pip install huggingface_hub
    from huggingface_hub import HfApi, login, create_repo
    from huggingface_hub.utils import HfHubHTTPError

# ============================================================================
# CONFIGURAÇÃO - AJUSTE AQUI
# ============================================================================
# Substitua pelos seus dados:
HF_REPO_ID = "seu-usuario/biobyia-medical-lora"  # Ex: "joao-silva/biobyia-medical-lora"
HF_TOKEN = None  # Ou coloque seu token aqui: "hf_xxxxxxxxxxxxx"
PRIVATE_REPO = False  # True para repositório privado

# ============================================================================
# UPLOAD
# ============================================================================
if HF_REPO_ID == "seu-usuario/biobyia-medical-lora":
    print("⚠️  ATENÇÃO: Configure HF_REPO_ID antes de fazer upload!")
    print("   Edite a variável HF_REPO_ID nesta célula")
    print("   Formato: seu-usuario/nome-do-modelo")
else:
    print(f"📤 Preparando upload para: {HF_REPO_ID}")
    
    # Login
    if HF_TOKEN:
        login(token=HF_TOKEN)
    else:
        print("🔐 Faça login no Hugging Face:")
        login()
    
    # Cria repositório
    api = HfApi()
    try:
        create_repo(
            repo_id=HF_REPO_ID,
            repo_type="model",
            private=PRIVATE_REPO,
            exist_ok=True
        )
        print(f"✅ Repositório criado/verificado: {HF_REPO_ID}")
    except HfHubHTTPError as e:
        if "already exists" in str(e).lower():
            print(f"✅ Repositório já existe: {HF_REPO_ID}")
        else:
            raise
    
    # Faz upload
    print(f"\n📤 Fazendo upload de: {MODEL_OUTPUT_DIR}")
    print("   Isso pode levar alguns minutos...")
    
    try:
        api.upload_folder(
            folder_path=str(MODEL_OUTPUT_DIR),
            repo_id=HF_REPO_ID,
            repo_type="model",
            ignore_patterns=["*.pt", "*.bin", "__pycache__", "*.pyc"]
        )
        print(f"\n✅ Upload concluído com sucesso!")
        print(f"   Modelo disponível em: https://huggingface.co/{HF_REPO_ID}")
        print(f"\n💡 Próximos passos:")
        print(f"   1. Acesse: https://huggingface.co/{HF_REPO_ID}")
        print(f"   2. Teste o modelo na interface do Hugging Face")
        print(f"   3. Use no backend com HuggingFaceProvider")
    except Exception as e:
        print(f"\n❌ Erro durante upload: {e}")
        print(f"\n💡 Alternativa: Use o script upload_to_huggingface.py localmente")
        print(f"   python upload_to_huggingface.py --lora_path {MODEL_OUTPUT_DIR} --repo_id {HF_REPO_ID}")


In [ ]:
# ============================================================================
# CÉLULA 13: CARREGAMENTO E TESTE DO MODELO SALVO
# ============================================================================
# Demonstra como carregar o modelo salvo e fazer inferência.
# Esta célula é útil para testar o modelo após reiniciar o ambiente.

print("=" * 80)
print("CARREGANDO MODELO SALVO")
print("=" * 80)

# Carrega modelo salvo
loaded_model, loaded_tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(MODEL_OUTPUT_DIR),
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)

FastLanguageModel.for_inference(loaded_model)

print("✅ Modelo carregado com sucesso!")

# Testa com outro exemplo
example_instruction = get_instruction_only()
example_input = """Contexto: Assessment of visual acuity depends on the optotypes used for measurement.
Pergunta: What are the differences between Landolt C and Snellen E acuity in strabismus amblyopia?"""

prompt = get_medical_alpaca_prompt(example_instruction, example_input, "")
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = loaded_tokenizer([prompt], return_tensors="pt").to(device)

# Usa TextStreamer para visualizar geração em tempo real
text_streamer = TextStreamer(loaded_tokenizer)
_ = loaded_model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=INFERENCE_CONFIG['max_new_tokens']
)
